In [17]:
import pandas as pd
import numpy as np

def inspection(data:list,reverse=False,min_num=0.5,max_num=3):
    if reverse:
        data = data[::-1]
    df  = data[0].copy()
    for i in data[1].iloc[:,0].values:
        if i in df.iloc[:,0].values:
            continue
        else:
            temp = pd.DataFrame([i],columns=[df.columns[0]])
            df = pd.concat([df,temp],axis=0)
    df = df.sort_values(by=[df.columns[0]]).interpolate()
    df = df.reset_index(drop=True)
    df = df[df.iloc[:,0].values>min_num]
    result = df[df.iloc[:,0].values<max_num]
    return result

In [10]:
data = pd.read_csv('../../data/all_position.csv')
cfd_index = np.arange(0,20,4)
sim_index = np.arange(2,20,4)
cfd,sim = [],[]
for i in range(len(cfd_index)):
    cfd.append(data.iloc[:,cfd_index[i]:cfd_index[i]+2])
    sim.append(data.iloc[:,sim_index[i]:sim_index[i]+2])

result = pd.DataFrame()
for i in range(len(cfd)):
    data = [cfd[i].dropna(),sim[i].dropna()]
    data[1] = data[1].drop(index=1)
    temp1 = inspection(data)
    temp2 = inspection(data,reverse=True)
    result = pd.concat([result,temp1,temp2],axis=1)

result.to_csv('../../data/position_inspection.csv',index=False)

In [18]:
data = pd.read_csv('../../data/all_data.csv')
cfd_index = np.arange(0,20,4)
sim_index = np.arange(2,20,4)
cfd,sim = [],[]
for i in range(len(cfd_index)):
    cfd.append(data.iloc[:,cfd_index[i]:cfd_index[i]+2])
    sim.append(data.iloc[:,sim_index[i]:sim_index[i]+2])

result = pd.DataFrame()
for i in range(len(cfd)):
    data = [cfd[i].dropna(),sim[i].dropna()]
    temp1 = inspection(data)
    temp2 = inspection(data,reverse=True)
    result = pd.concat([result,temp1,temp2],axis=1)

# result.to_csv('../../data/inspection.csv',index=False)

In [19]:
# 均方根误差，RMSE
rmse_index = np.arange(1,20,2)
rmse_data = [result.iloc[:,i] for i in rmse_index]
index = np.arange(0,10,2)
for i in index:
    a = np.asarray(rmse_data[i].dropna())
    b = np.asarray(rmse_data[i+1].dropna())
    print(np.sqrt(np.mean((a-b)**2)))

125.41483382061983
93.10384717650838
70.29404607504206
162.58443370218677
229.2266467153651


In [20]:
# 平均百分比误差，MAPE
temp = []
for i in index:
    a = np.asarray(rmse_data[i].dropna())
    b = np.asarray(rmse_data[i+1].dropna())
    temp.append(np.mean(abs((a-b)/a)))

test = pd.DataFrame(temp).T
test

,0,1,2,3,4
0,0.137728,0.11018,0.085405,0.160489,0.205731
